In [476]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys

## Pair sound files and annotation
# Combine the two cells to avoid overlap!!!

In [477]:
soundfiles = glob.glob('trainfiles/**/*.sph', recursive = True)
#annofiles = glob.glob('./**/*.txt', recursive = True) + glob.glob('./**/*.sgml', recursive = True)
data = {}
for soundfile in soundfiles:
    name = soundfile.split('/')[-1].split('.')[0]
    annofile = list(glob.glob('trainfiles/**/'+name+'.txt', recursive = True))+list(glob.glob('trainfiles/**/'+name+'.sgml', recursive = True))
    annofile = annofile[0]
    data[soundfile] = annofile
start_load = time.time()
soundfiles = list(data.keys())
data

{'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960521_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960521_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960522_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960522_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960523_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960523_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960524_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960524_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960528_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960528_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960530_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960530_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960604_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960604_.txt',
 'trainfiles/LDC97S44/H4E96_01/h4eng_sp/a960605_.sph': 'trainfiles/LDC97T22/hub4_eng_train_trans/aABC_NLI/a960605_.txt',
 'trainfiles/LDC97S44/H4E96_01/h

## Figure out gender of each speaker

In [68]:
# gender = {}
# for annofile in data.values():
#     doc = parse(annofile)
#     for tag in doc.getroot().iter():
#         if tag.tag in ['segment', 'turn']:
#             if 'spkrtype' in tag.attrib:
#                 gender[tag.attrib['speaker']] = tag.attrib['spkrtype']
gender = np.load('gender.npy', allow_pickle = True).reshape(1)[0]

# Save Segments

In [402]:
def calculate_overlap(row1, row2):
    if row1['soundfile'] != row2['soundfile']: # can only overlap if in same file
        return 0
    if row1['end']<row2['start'] or row1['start']>row2['end']:
        return 0
    if row1['start']<row2['start']:
        return row1['end']-row2['start']
    return row2['end']-row1['start']
def getstart(segment):
    return float(segment['s_time']) if segment.has_attr('s_time') else float(segment['starttime'])
def getend(segment):
    return float(segment['e_time']) if segment.has_attr('e_time') else float(segment['endtime'])
end_df = pd.DataFrame()
start_df = pd.DataFrame()
for i, soundfile in enumerate(soundfiles):
    name = soundfile.split('/')[-1].split('.')[0] # name of soundfile
    print(i)
    sph = SPHFile(soundfile)
    sound, sr = sph.content, sph.format['sample_rate']
    annofile = data[soundfile]
    with open(annofile) as file:
        soup = BeautifulSoup(file,'html.parser')
    tags = soup.find_all(['segment', 'turn'])
    for j, segment in enumerate(tags):
        save_start, save_end = True, True
        text = segment.content
        end = getend(segment)
        start = getstart(segment)
        speaker = segment['speaker'].lower()
        if speaker[:4] in ['male', 'fema', 'spkr']:
            speaker = name+'_'+speaker
        if speaker.lower().find('announcer') != -1:
            #print(speaker)
            continue
        if speaker in ['01janedoe', '01johndoe', '02janedoe', '02johndoe', '04johndoe', 'anchor1', 'unison']:
            continue
        if end > len(sound)/sr: # if end exceeds length, skip
            continue
        if start >= end - 2: # if segment too short
            continue
            
        
        # ending segments
        if j + 1 < len(tags) and getstart(tags[j+1]) < end: # if someone interrupted talking
            #print('end interrupted')
            save_end = False
        end_idx = int(end*sr)
        start_idx = int((end-2)*sr) # truncate segment to 2s
        end_segment = sound[start_idx:end_idx]
        seg_object = audiosegment.from_numpy_array(end_segment, sr)
        silences = pydub.silence.detect_silence(seg_object, min_silence_len = 100, silence_thresh=-32, seek_step = 30)
        silences = [[int(a/1000*sr),int(b/1000*sr)] for [a, b] in silences] # turn unit into samples
        if silences!= []:
            if silences[-1][1] == 2*sr: # if silence at end, remove that silence
                #print('readjusted end')
                end_idx = end_idx - 2*sr + silences[-1][0] # readjust silence end
                # reset start idx
                start_idx = int(start*sr)
                if start_idx >= end_idx - 2*sr: # if segment too short
                    continue
                start_idx = end_idx - 2*sr 
        # recrop
        end_segment = sound[start_idx:end_idx]
        seg_object = audiosegment.from_numpy_array(end_segment, sr)
        # recalculate silence
        silences = pydub.silence.detect_silence(seg_object, min_silence_len = 100, silence_thresh=-32, seek_step = 30)
        silences = [[int(a/1000*sr),int(b/1000*sr)] for [a, b] in silences]
        length_silence = sum([b-a for [a, b] in silences])
        if length_silence/len(end_segment) > 0.3:
            save_end = False
        end_filename = 'trainfiles/end_segments/'+name+'_seg'+str(j)+'.npy'
        end_row = {'soundfile': soundfile, 'segment_idx': j, 'start': start, 'end': end, 
               'silence_ratio': length_silence/len(end_segment), 'speaker':speaker, 'segfile':end_filename}
        
        
        # starting segments
        if j > 0 and getend(tags[j-1]) > start: #if someone interrupted talking
            #print('start interrupted')
            save_start = False
        end_idx = int((start+2)*sr)
        start_idx = int(start*sr) # truncate segment to 2s
        start_segment = sound[start_idx:end_idx]
        seg_object = audiosegment.from_numpy_array(start_segment, sr)
        silences = pydub.silence.detect_silence(seg_object, min_silence_len = 100, silence_thresh=-32, seek_step = 30)
        silences = [[int(a/1000*sr),int(b/1000*sr)] for [a, b] in silences] # turn unit into samples
        if silences!= []:
            if silences[0][0] == 0: # if silence at start, remove that silence # different
                #print('readjusted start')
                start_idx = start_idx + silences[0][1] - int(0.1*sr) # readjust silence start # different, include attack
                # reset end idx
                end_idx = int(end*sr)
                if start_idx >= end_idx - 2*sr: # if segment too short
                    continue
                end_idx = start_idx + 2*sr # different
        # recrop
        start_segment = sound[start_idx:end_idx]
        seg_object = audiosegment.from_numpy_array(start_segment, sr)
        # recalculate silence
        silences = pydub.silence.detect_silence(seg_object, min_silence_len = 100, silence_thresh=-32, seek_step = 30)
        silences = [[int(a/1000*sr),int(b/1000*sr)] for [a, b] in silences]
        length_silence = sum([b-a for [a, b] in silences])
        if length_silence/len(start_segment) > 0.3:
            save_start = False
        start_filename = 'trainfiles/start_segments/'+name+'_seg'+str(j)+'.npy'
        start_row = {'soundfile': soundfile, 'segment_idx': j, 'start': start, 'end': end, 
                   'silence_ratio': length_silence/len(start_segment), 'speaker':speaker, 'segfile':start_filename}
        if save_end and save_start and calculate_overlap(start_row, end_row)>sr: # if overlap, only take one
            if np.random.randn()>0:
                save_end = False
            else:
                save_start = False
        #print(save_end, save_start)
        if save_end:
            np.save(end_filename, end_segment)
            end_df = end_df.append(end_row, ignore_index = True)
        if save_start:
            np.save(start_filename, start_segment)
            start_df = start_df.append(start_row, ignore_index = True)
end_df.to_csv('trainfiles/end.csv', index = False)
start_df.to_csv('trainfiles/start.csv', index = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Check for redundant speaker

In [544]:
segments_df = start_df.append(end_df, ignore_index = True)
#spkr_df = segments_df[segments_df['speaker'] == 'j960522b_johndoe001']
spkr_df = segments_df[segments_df['speaker'].str.contains('jack_smith')]
idx = np.random.randint(len(spkr_df))
segfile = spkr_df.iloc[idx]['segfile']
seg = np.fromfile(segfile, dtype = np.int16)
print(spkr_df.iloc[idx]['soundfile'], spkr_df.iloc[idx]['speaker'], spkr_df.iloc[idx]['segfile'])
print(len(Counter(segments_df['speaker'])), ' speakers')
ipd.Audio(seg, rate = sr)
#rob_gifford, bill_clinton, jack_smith

trainfiles/LDC97S44/H4E96_06/h4eng_sp/c960526_.sph jack_smith trainfiles/end_segments/c960526__seg54.npy
5253  speakers


In [478]:
from bs4 import BeautifulSoup
total_segs = 0
soundfile = soundfiles[200]
name = soundfile.split('/')[-1].split('.')[0] # name of soundfile
sph = SPHFile(soundfile)
sound, sr = sph.content, sph.format['sample_rate']
annofile = data[soundfile]
with open(annofile) as file:
    soup = BeautifulSoup(file,'html.parser')
soup.find_all(['segment', 'turn'])

[<turn endtime="9.703" speaker="Art_Harris" spkrtype="male" starttime="4.660">
 <time sec="4.660">
  He remains here, behind locked doors, guarded by police twenty-four hours a day.
 </time></turn>,
 <turn endtime="12.635" speaker="Art_Harris" spkrtype="male" starttime="9.703">
 <time sec="9.703">
  ^Art ^Harris, _C_N_N ^Paris.
 </time></turn>,
 <turn endtime="33.824" speaker="Leon_Harris" spkrtype="male" starttime="12.635">
 <time sec="12.635">
  If you think those warnings that ^Redux and ^Fen-phen can cause serious
 <time sec="16.199">
  heart damage are enough to steer anyone away from those diet drugs, {breath}
 <time sec="19.391">
  think again. It seems, for some, the desire to drop the pounds
 <time sec="22.635">
  far outweighs the dramatic risks. {breath}
 <time sec="24.873">
  Tonight, we take a closer look at ^America's obsession with fighting fat
 <time sec="28.513">
  and the push to make drugs safer. {breath}
 <time sec="30.637">
  Our coverage begins now with _C_N_N's ^

In [345]:
from bs4 import BeautifulSoup
total_segs = 0
for i, soundfile in enumerate(soundfiles):
    name = soundfile.split('/')[-1].split('.')[0] # name of soundfile
    sph = SPHFile(soundfile)
    sound, sr = sph.content, sph.format['sample_rate']
    annofile = data[soundfile]
    try:
        with open(annofile) as file:
            soup = BeautifulSoup(file,'html.parser')
        print(i, annofile, len(soup.find_all(['segment', 'turn'])))
        total_segs+=len(soup.find_all(['segment', 'turn']))
        
    except:
        print('broken file ', annofile)
    if len(soup.find_all(['segment', 'turn'])) < 10:
        print(soup.find_all(['segment', 'turn']))
        print('error parsing ', annofile)
        break
total_segs

0 trainfiles/LDC97T22/hub4_eng_train_trans/dCNN_EED/d960531a.txt 97
1 trainfiles/LDC97T22/hub4_eng_train_trans/dCNN_EED/d960604b.txt 69
2 trainfiles/LDC97T22/hub4_eng_train_trans/dCNN_EED/d960531b.txt 92
3 trainfiles/LDC97T22/hub4_eng_train_trans/dCNN_EED/d960604a.txt 92
4 trainfiles/LDC97T22/hub4_eng_train_trans/gCNN_PRN/g960523_.txt 92
5 trainfiles/LDC97T22/hub4_eng_train_trans/hCNN_TWT/h960516_.txt 157
6 trainfiles/LDC97T22/hub4_eng_train_trans/gCNN_PRN/g960529_.txt 70
7 trainfiles/LDC97T22/hub4_eng_train_trans/gCNN_PRN/g960611_.txt 89
8 trainfiles/LDC97T22/hub4_eng_train_trans/gCNN_PRN/g960607_.txt 88
9 trainfiles/LDC97T22/hub4_eng_train_trans/hCNN_TWT/h960517_.txt 150
10 trainfiles/LDC97T22/hub4_eng_train_trans/gCNN_PRN/g960524_.txt 75
11 trainfiles/LDC97T22/hub4_eng_train_trans/jNPR_ATC/j960531d.txt 78
12 trainfiles/LDC97T22/hub4_eng_train_trans/jNPR_ATC/j960607c.txt 79
13 trainfiles/LDC97T22/hub4_eng_train_trans/jNPR_ATC/j960613c.txt 82
14 trainfiles/LDC97T22/hub4_eng_train_tran

KeyboardInterrupt: 

In [284]:
spkrfile = 'trainfiles/LDC97T22/hub4_eng_train_trans/spkrlist.sgml'
spkrlist = []
with open(spkrfile) as file:
    for line in file:
        soup = BeautifulSoup(line, 'html.parser')
        doc = soup.find_all('speaker')
        if len(doc):
            doc = doc[0]
            spkrlist.append(doc['name'])
len(spkrlist)

3116

In [285]:
print(len(spkrlist))
spkrlist = Counter(spkrlist)
print([speaker for speaker in spkrlist.keys() if spkrlist[speaker]>1])
print(len(spkrlist))

3116
['Alfonse_DAmato', 'Amanda_Greenleaf_Whelan', 'Ann_Lewis', 'Anne_McDermott', 'Bo_Gritz', 'Brian_Jenkins', 'Brent_Sadler', 'Kathy_Lohr', 'Oliver_Caman', 'Dan_Rutz', 'Don_Knapp', 'Reni_Vaughn', 'Lisa_Price', 'John_Ydstie', 'Steve_Inskeep', 'Jamie_McIntyre', 'Jim_Zarroli', 'Katharine_Barrett', 'Kent_Ninomiya', 'Lauch_Faircloth', 'Martha_Raddatz', 'Martin_Buser', 'Mary_Schiavo', 'Michael_Sivy', 'Philip_Boroff', 'Roger_Cossack', 'William_Perry', 'Sherry_Matteucci', 'William_Clinger']
3087


# Train&Validate Split, figure out speakers

## Pool start/end segments

In [294]:
start_df = pd.read_csv('trainfiles/start.csv')
end_df = pd.read_csv('trainfiles/end.csv')
start_df = start_df.sample(frac=1).reset_index(drop=True)
end_df = end_df.sample(frac=1).reset_index(drop=True)
speakers = set(list(start_df['speaker'])+list(end_df['speaker']))
print(len(speakers), 'speakers')

5253 speakers


## Trainset

In [307]:
# sample training data
start_df_train = start_df[:int(len(start_df)*0.9)]
end_df_train = end_df[:int(len(end_df)*0.9)]
segments_df_train = start_df_train.append(end_df_train, ignore_index = True)
segments_by_speaker_train = {}
# for faster fetching of triplets negative speaker
for speaker in speakers:
    segments_by_speaker_train[speaker] = segments_df_train[segments_df_train['speaker'] == speaker]
# for faster fetching of pairs negative speaker
start_by_speaker_train = {}
for speaker in speakers:
    start_by_speaker_train[speaker] = start_df_train[start_df_train['speaker'] == speaker]
end_by_speaker_train = {}
for speaker in speakers:
    end_by_speaker_train[speaker] = end_df_train[end_df_train['speaker'] == speaker]

In [309]:
speakers_train = [speaker for speaker in speakers if len(segments_by_speaker_train[speaker])>0]
speakers_start_train = [speaker for speaker in speakers if len(start_by_speaker_train[speaker])>0]
speakers_end_train = [speaker for speaker in speakers if len(end_by_speaker_train[speaker])>0]
len(speakers_train), len(speakers_start_train), len(speakers_end_train)

(5143, 4497, 4512)

## Make triplets

In [306]:
triplets = []
for _, anchor_speaker in enumerate(speakers_train):
    print(_)
    negative_speakers = [speaker for speaker in speakers_train if speaker != anchor_speaker]
    anchor_segments = segments_by_speaker_train[anchor_speaker]
    anchor_indices = np.arange(len(anchor_segments))
    positive_indices = np.arange(len(anchor_segments))
    n = min(len(anchor_segments), 35)
    np.random.shuffle(anchor_indices)# shuffle the anchor segments
    for i in range(n): # iterate over anchors
        anchor_segment = anchor_segments.iloc[anchor_indices[i]]
        np.random.shuffle(positive_indices)# shuffle the anchor segments
        for j in range(n): # iterate over pos-neg triplets
            positive_segment = anchor_segments.iloc[positive_indices[j]]
            negative_speaker = negative_speakers[np.random.randint(0, len(negative_speakers))]
            negative_segments = segments_by_speaker_train[negative_speaker] # segments for a specific random speaker
            negative_segment = negative_segments.iloc[np.random.randint(0, len(negative_segments))]
            row = {'anchor_speaker': anchor_speaker, 'anchor_file': anchor_segment['segfile'], 
                   'positive_file': positive_segment['segfile'], 'negative_speaker': negative_speaker,
                   'negative_file': negative_segment['segfile']}
            triplets.append(row)
triplets = pd.DataFrame(triplets)
triplets.to_csv('trainfiles/train-triplets.csv')
triplets

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074


3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735


,anchor_speaker,anchor_file,positive_file,negative_speaker,negative_file
0,f960606a_m_us_026,trainfiles/end_segments/f960606a_seg21.npy,trainfiles/start_segments/f960606a_seg21.npy,file2_janedoe001,trainfiles/end_segments/file2_seg25.npy
1,f960606a_m_us_026,trainfiles/end_segments/f960606a_seg21.npy,trainfiles/end_segments/f960606a_seg21.npy,em970927_spkr_12,trainfiles/end_segments/em970927_seg68.npy
2,f960606a_m_us_026,trainfiles/start_segments/f960606a_seg21.npy,trainfiles/start_segments/f960606a_seg21.npy,steven_lamb,trainfiles/end_segments/eh971016_seg151.npy
3,f960606a_m_us_026,trainfiles/start_segments/f960606a_seg21.npy,trainfiles/end_segments/f960606a_seg21.npy,k960606_m_us_004,trainfiles/end_segments/k960606__seg51.npy
4,eo971225_spkr_22,trainfiles/end_segments/eo971225_seg146.npy,trainfiles/end_segments/eo971225_seg146.npy,rose_johnson,trainfiles/start_segments/b960611a_seg14.npy
...,...,...,...,...,...
390234,eh971015_spkr_10,trainfiles/end_segments/eh971015_seg91.npy,trainfiles/end_segments/eh971015_seg91.npy,ray_mabus,trainfiles/start_segments/a960625__seg81.npy
390235,eh971015_spkr_10,trainfiles/start_segments/eh971015_seg91.npy,trainfiles/end_segments/eh971015_seg91.npy,c960517_m_us_008,trainfiles/start_segments/c960517__seg28.npy
390236,eh971015_spkr_10,trainfiles/start_segments/eh971015_seg91.npy,trainfiles/start_segments/eh971015_seg91.npy,kathleen_list,trainfiles/end_segments/a960606__seg34.npy
390237,i960605__male_native1,trainfiles/start_segments/i960605__seg22.npy,trainfiles/start_segments/i960605__seg22.npy,ee970807_spkr_27,trainfiles/start_segments/ee970807_seg157.npy


## Make pairs

In [324]:
pairs = []
for _, anchor_speaker in enumerate(speakers_train):
    print(_)
    if anchor_speaker not in speakers_start_train or anchor_speaker not in speakers_end_train:
        continue # if speaker doesn't have both a start&end segment
    negative_speakers = [speaker for speaker in speakers_start_train if speaker != anchor_speaker]
    anchor_segments = end_by_speaker_train[anchor_speaker] # at end of segment
    positive_segments = start_by_speaker_train[anchor_speaker] # at start of segment
    anchor_indices = np.arange(len(anchor_segments))
    positive_indices = np.arange(len(positive_segments))
    np.random.shuffle(anchor_indices)# shuffle the anchor segments
    n_s = min(len(anchor_segments), 50)
    n_p = min(len(positive_segments), 50)
    for i in range(n_s): #iterate over anchors
        anchor_segment = anchor_segments.iloc[anchor_indices[i]]
        np.random.shuffle(positive_indices)
        for j in range(n_p): #iterate over compared segment
            # positive examples
            positive_segment = positive_segments.iloc[positive_indices[j]]
            pos_row = {'first_speaker': anchor_speaker, 'first_file': anchor_segment['segfile'], 
                       'second_speaker': anchor_speaker, 'second_file': positive_segment['segfile'],
                       'label': 0} # no speaker change
            # negative examples
            negative_speaker = negative_speakers[np.random.randint(0, len(negative_speakers))]
            negative_segments = start_by_speaker_train[negative_speaker] # segments for a specific random speaker
            negative_segment = negative_segments.iloc[np.random.randint(0, len(negative_segments))]
            neg_row = {'first_speaker': anchor_speaker, 'first_file': anchor_segment['segfile'], 
               'second_speaker': negative_speaker, 'second_file': negative_segment['segfile'],
               'label': 1} # has speaker change
            pairs.append(pos_row)
            pairs.append(neg_row)
pairs = pd.DataFrame(pairs)
pairs.to_csv('trainfiles/train-pairs.csv')
pairs

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094


3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745


,first_speaker,first_file,second_speaker,second_file,label
0,f960606a_m_us_026,trainfiles/end_segments/f960606a_seg21.npy,f960606a_m_us_026,trainfiles/start_segments/f960606a_seg21.npy,0
1,f960606a_m_us_026,trainfiles/end_segments/f960606a_seg21.npy,j960520b_f_us_004,trainfiles/start_segments/j960520b_seg49.npy,1
2,frank_tolsey,trainfiles/end_segments/b960605c_seg47.npy,frank_tolsey,trainfiles/start_segments/b960605c_seg47.npy,0
3,frank_tolsey,trainfiles/end_segments/b960605c_seg47.npy,ron_claiborne,trainfiles/start_segments/ea980112_seg7.npy,1
4,frank_tolsey,trainfiles/end_segments/b960605c_seg47.npy,frank_tolsey,trainfiles/start_segments/b960605c_seg53.npy,0
...,...,...,...,...,...
503687,h960522_m_us_004,trainfiles/end_segments/h960522__seg18.npy,andrea_dubrovsky,trainfiles/start_segments/eh971029_seg66.npy,1
503688,ed980106_spkr_2,trainfiles/end_segments/ed980106_seg6.npy,ed980106_spkr_2,trainfiles/start_segments/ed980106_seg6.npy,0
503689,ed980106_spkr_2,trainfiles/end_segments/ed980106_seg6.npy,em971230_spkr_4,trainfiles/start_segments/em971230_seg10.npy,1
503690,eh971015_spkr_10,trainfiles/end_segments/eh971015_seg91.npy,eh971015_spkr_10,trainfiles/start_segments/eh971015_seg91.npy,0


## Val set

In [325]:
start_df_val = start_df[int(len(start_df)*0.9):]
end_df_val = end_df[int(len(end_df)*0.9):]
segments_df_val = start_df_val.append(end_df_val, ignore_index = True)
segments_by_speaker_val = {}
# for faster fetching of triplets negative speaker
for speaker in speakers:
    segments_by_speaker_val[speaker] = segments_df_val[segments_df_val['speaker'] == speaker]
# for faster fetching of pairs negative speaker
start_by_speaker_val = {}
for speaker in speakers:
    start_by_speaker_val[speaker] = start_df_val[start_df_val['speaker'] == speaker]
end_by_speaker_val = {}
for speaker in speakers:
    end_by_speaker_val[speaker] = end_df_val[end_df_val['speaker'] == speaker]

In [326]:
speakers_val = [speaker for speaker in speakers if len(segments_by_speaker_val[speaker])>0]
speakers_start_val = [speaker for speaker in speakers if len(start_by_speaker_val[speaker])>0]
speakers_end_val = [speaker for speaker in speakers if len(end_by_speaker_val[speaker])>0]
len(speakers_val), len(speakers_start_val), len(speakers_end_val)

(1617, 1042, 1009)

## Make pairs

In [327]:
pairs = []
for _, anchor_speaker in enumerate(speakers_val):
    print(_)
    if anchor_speaker not in speakers_start_val or anchor_speaker not in speakers_end_val:
        continue # if speaker doesn't have both a start&end segment
    negative_speakers = [speaker for speaker in speakers_start_val if speaker != anchor_speaker]
    anchor_segments = end_by_speaker_val[anchor_speaker] # at end of segment
    positive_segments = start_by_speaker_val[anchor_speaker] # at start of segment
    anchor_indices = np.arange(len(anchor_segments))
    positive_indices = np.arange(len(positive_segments))
    np.random.shuffle(anchor_indices)# shuffle the anchor segments
    n_s = min(len(anchor_segments), 50)
    n_p = min(len(positive_segments), 50)
    for i in range(n_s): #iterate over anchors
        anchor_segment = anchor_segments.iloc[anchor_indices[i]]
        np.random.shuffle(positive_indices)
        for j in range(n_p): #iterate over compared segment
            # positive examples
            positive_segment = positive_segments.iloc[positive_indices[j]]
            pos_row = {'first_speaker': anchor_speaker, 'first_file': anchor_segment['segfile'], 
                       'second_speaker': anchor_speaker, 'second_file': positive_segment['segfile'],
                       'label': 0} # no speaker change
            # negative examples
            negative_speaker = negative_speakers[np.random.randint(0, len(negative_speakers))]
            negative_segments = start_by_speaker_val[negative_speaker] # segments for a specific random speaker
            negative_segment = negative_segments.iloc[np.random.randint(0, len(negative_segments))]
            neg_row = {'first_speaker': anchor_speaker, 'first_file': anchor_segment['segfile'], 
               'second_speaker': negative_speaker, 'second_file': negative_segment['segfile'],
               'label': 1} # has speaker change
            pairs.append(pos_row)
            pairs.append(neg_row)
pairs = pd.DataFrame(pairs)
pairs.to_csv('trainfiles/val-pairs.csv')
pairs

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,first_speaker,first_file,second_speaker,second_file,label
0,ee970701_spkr_7,trainfiles/end_segments/ee970701_seg100.npy,ee970701_spkr_7,trainfiles/start_segments/ee970701_seg100.npy,0
1,ee970701_spkr_7,trainfiles/end_segments/ee970701_seg100.npy,i960604_f_us_001,trainfiles/start_segments/i960604__seg369.npy,1
2,peter_kenyon,trainfiles/end_segments/j960522a_seg18.npy,peter_kenyon,trainfiles/start_segments/j960522c_seg12.npy,0
3,peter_kenyon,trainfiles/end_segments/j960522a_seg18.npy,liz_reyes,trainfiles/start_segments/b960618__seg48.npy,1
4,peter_kenyon,trainfiles/end_segments/j960614b_seg3.npy,peter_kenyon,trainfiles/start_segments/j960522c_seg12.npy,0
...,...,...,...,...,...
39203,jeanne_moos,trainfiles/end_segments/eo971226_seg175.npy,terry_moran,trainfiles/start_segments/ea980108_seg11.npy,1
39204,jeanne_moos,trainfiles/end_segments/eo971226_seg175.npy,jeanne_moos,trainfiles/start_segments/ee970806_seg180.npy,0
39205,jeanne_moos,trainfiles/end_segments/eo971226_seg175.npy,fred_katayama,trainfiles/start_segments/ee970806_seg96.npy,1
39206,jeanne_moos,trainfiles/end_segments/eo971226_seg175.npy,jeanne_moos,trainfiles/start_segments/eo971226_seg196.npy,0
